# Data Preprocessing
in this notebook, We will clean the Arsarcasmv2 Arabic sarcasm detection dataset and prepare it to be ready to build training models. We will perform the following operations:

*   Remove mentions (user name)
*   Remove URLs and links
*   Remove emojis and emotions
*   Remove arabic and english punctuations
*   Remove diacritics
*   Remove different symbols
*   Remove digits
*   Remove stopwords
*   apply stemming


we will apply that for training and testing datasets.





In [ ]:
# install needed modules
! pip install tashaphyne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 15.6 MB/s eta 0:00:00


In [ ]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import modules
import matplotlib.pyplot as plt
import seaborn as sn
import os
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from tashaphyne.stemming import ArabicLightStemmer
from sklearn.preprocessing import LabelEncoder

In [ ]:
# read data from csv file
data_train = pd.read_csv('/content/drive/MyDrive/sarcasm detection/ArSarcasm-v2/training_data.csv')
data_test = pd.read_csv('/content/drive/MyDrive/sarcasm detection/ArSarcasm-v2/testing_data.csv')

In [ ]:
# show the first 20 rows
data_train.head(20)

,tweet,sarcasm,sentiment,dialect
0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa
1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa
2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa
3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf
4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa
5,"""طبيب #المقاصة: إصابة #أحمد_الشيخ بسيطة تحتاج ...",False,NEG,msa
6,مرسي : مش هنام اكتر من اربع ساعات في اليوم.......,True,NEG,egypt
7,#انتخبوا_البرص مشكلة,True,NEG,egypt
8,"""Retweeted هاشتاق العرب (@TheArabHash):#فيديو ...",False,POS,msa
9,▪️محافظ العاصمة تفقد سير العملية الانتخابية في...,False,NEU,msa


In [ ]:
# calculate null values in each column
data_train.isnull().sum()

tweet        0
sarcasm      0
sentiment    0
dialect      0
dtype: int64

In [ ]:
data_test.isnull().sum()

tweet        0
sarcasm      0
sentiment    0
dialect      0
dtype: int64

In [ ]:
# calculate number of positive and negative reviews
data_train['sarcasm'].value_counts()

False    10380
True      2168
Name: sarcasm, dtype: int64

In [ ]:
# create function to calculate the length for each review
def compte_mots(phrase):
    return len(phrase.split())

# create new feature after apply function on dataset
data_train["len_sentences"] = data_train["tweet"].apply(compte_mots)

In [ ]:
# show the maximum review length
print("maximum length for sentence is: {}".format(max(data_train.len_sentences)))

maximum length for sentence is: 61


In [ ]:
# Create a function for preprocessing Arabic text
def preprocessing(x):

    # Remove mentions (usernames) from the text
    x = re.sub('@[^\s]+', ' ', x)

    # Remove URLs and links from the text
    x = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', x)

    # Define a pattern to match emojis and emoticons
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # Chinese characters
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030""]+", flags=re.UNICODE)
    # Remove emojis and emoticons from the text
    x = emoji_pattern.sub(r'', x)

    # Define Arabic and English punctuation marks
    ar_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ#'''
    en_punctuations = string.punctuation

    # Combine Arabic and English punctuation marks
    punctuations = ar_punctuations + en_punctuations

    # Remove punctuations from the text
    x = x.translate(str.maketrans('', '', punctuations))

    # Define a pattern to match Arabic diacritics
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    # Remove Arabic diacritics from the text
    x = re.sub(arabic_diacritics, '', str(x))

    # Return the preprocessed text
    return x

In [ ]:
# apply preprocessing function on reviews
data_train["cleaned_tweet"] = data_train["tweet"].apply(lambda x: preprocessing(x))
data_test["cleaned_tweet"] = data_test["tweet"].apply(lambda x: preprocessing(x))

In [ ]:
# show the first 30 rows
data_train['cleaned_tweet'].head(30)

0     د محمودالعلايليأرى أن الفريق أحمدشفيق رقم مهم ...
1                            مع فيدرر يا آجا والكبار   
2     الداعون لمبدأ الاختلاط بين الجنسين كالداعين لإ...
3                   مساكين من الصبح و هوما رايحين را...
4     قل شرق حلب ولا تقل حلب الشرقية وقل غرب حلب ولا...
5     طبيب المقاصة إصابة أحمدالشيخ بسيطة تحتاج للراح...
6     مرسي  مش هنام اكتر من اربع ساعات في اليوم لو ن...
7                                    انتخبواالبرص مشكلة
8     Retweeted هاشتاق العرب    أهالي القدس يرفعون ا...
9     محافظ العاصمة تفقد سير العملية الانتخابية في ع...
10    الاتجاه أوباما يصل الى إسرائيل للمشاركة بدفن ب...
11              والله ما عليش مايمدي ايفون سبع يا حبيبي
12                                               اف هذا
13    ريال مدريد  كلوب أميركا نصف النهائيالساعه ١٣٠ ...
14        وقت الشدة في ظنتي الجيش المصري حيوقف مع ال...
15    عمليات الأنبار تحرر جزيرة هيت بالكامل من مجرمي...
16    القطريين اختتطفو لانهم من الاسره الحاكمه وليس ...
17    انت لسه مبقتش من شركاء اوبر  الفرصه لسه فأ

In [ ]:
# drop columns that we dont need
data_train.drop('len_sentences', axis=1, inplace=True)
data_train.drop('tweet', axis=1, inplace=True)
data_train.reset_index(drop=True, inplace=True)
data_test.drop('tweet', axis=1, inplace=True)
data_test.reset_index(drop=True, inplace=True)

# show the first 5 rows
data_train.head()

,sarcasm,sentiment,dialect,cleaned_tweet
0,False,NEU,msa,د محمودالعلايليأرى أن الفريق أحمدشفيق رقم مهم ...
1,False,NEU,msa,مع فيدرر يا آجا والكبار
2,True,NEG,msa,الداعون لمبدأ الاختلاط بين الجنسين كالداعين لإ...
3,True,NEG,gulf,مساكين من الصبح و هوما رايحين را...
4,False,NEU,msa,قل شرق حلب ولا تقل حلب الشرقية وقل غرب حلب ولا...


In [ ]:
data_test.head()

,sarcasm,sentiment,dialect,cleaned_tweet
0,False,NEG,msa,اخوي حانق يالغلا وشفيك معصب عادي تراهم بشر يفط...
1,True,NEG,msa,اف مو متعوده عليهم سته
2,False,POS,msa,اللهم اشف مرضانا ومرضى المسلمين
3,False,POS,gulf,ابشركم طلقت السات
4,True,NEG,msa,مؤشر خطير ٩٠٪ من الشخصيات البرلمانية في الكويت...


In [ ]:
# Download Arabic stopwords if not already downloaded
nltk.download('stopwords')

# Create a set of Arabic stopwords
stop_words = set(stopwords.words('arabic'))

# Create an Arabic stemmer
stemmer = ArabicLightStemmer()

# Define a function to preprocess text
def preprocess_text2(text):
    # Remove digits from the text
    text = re.sub(r'\d+', '', text)

    # Remove non-alphanumeric characters from the text
    text = re.sub(r'[^\w\s]', '', text)

    # remove englis characters
    text = re.sub(r'[a-zA-Z]', '', text)

    # Split the text into words
    words = text.split()

    # Apply stemming to each word and remove stopwords
    words = [stemmer.light_stem(word) for word in words if word not in stop_words]

    # Join the preprocessed words back into a text
    return ' '.join(words)

# Apply the preprocessing function to the 'cleaned_text' column in 'data' DataFrame
data_train['cleaned_tweet'] = data_train['cleaned_tweet'].apply(preprocess_text2)
data_test['cleaned_tweet'] = data_test['cleaned_tweet'].apply(preprocess_text2)

# Display the first 5 rows of the preprocessed data
data_train.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,sarcasm,sentiment,dialect,cleaned_tweet
0,False,NEU,msa,محمودالعلايليأرى فريق حمدشفيق رقم مهم معادل سي...
1,False,NEU,msa,درر آجا كبار
2,True,NEG,msa,داع مبدأ اختلاط جنس داع إلغاء تسعير ريد تصف سو...
3,True,NEG,gulf,مساك صبح هوم رايح راجع عاى غوغل عبت دال هههه
4,False,NEU,msa,قل شرق حلب قل حلب شرق قل غرب حلب قل حلب غرب حل...


In [ ]:
# schuffle dataset
data_train = data_train.sample(frac=1, random_state=42)

In [ ]:
# calculate null values in each column
data_train.isnull().sum()

sarcasm          0
sentiment        0
dialect          0
cleaned_tweet    0
dtype: int64

In [ ]:
# calculate null values in each column
data_test.isnull().sum()

sarcasm          0
sentiment        0
dialect          0
cleaned_tweet    0
dtype: int64

In [ ]:
data_train.dtypes

sarcasm            bool
sentiment        object
dialect          object
cleaned_tweet    object
dtype: object

In [ ]:
data_train.reset_index(drop=True, inplace=True)
data_test.reset_index(drop=True, inplace=True)

In [ ]:
# save cleaned data train and test in csv files
data_train.to_csv('/content/drive/MyDrive/sarcasm detection/ArSarcasm-v2/cleaned_sarcasam_train.csv')
data_test.to_csv('/content/drive/MyDrive/sarcasm detection/ArSarcasm-v2/cleaned_sarcasam_test.csv')